In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data',header=None)

In [3]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
df.shape

(569, 32)

In [5]:
df[1]=df[1].replace({'M':1,'B':0})

C:\Users\Kelvi\AppData\Local\Temp\ipykernel_22972\417157543.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[1]=df[1].replace({'M':1,'B':0})


In [6]:
train_set=df.iloc[0:300]
val_set=df.iloc[300:500]

In [7]:
X=train_set[[2,3,4,5,6,7,8,9,10]]
y=val_set[1]
X_val=val_set[[2,3,4,5,6,7,8,9,10]]
y_val=val_set[1]

In [8]:
class early_stoping:
    def __init__(self,batch_size=30,lr=0.001,epochs=1000,patience=5,delta=0.001):
        self.batch_size=batch_size
        self.lr=lr
        self.epochs=epochs
        self.weight=None
        self.bias=None
        self.loss_history=[]
        self.wait=0
        self.patience=patience
        self.delta=delta
        self.best_weight_v_set=None
        self.best_bias_v_set=None
        self.best_loss=None
        self.val_loss_history=[]
        self.early_stop=False

    def log_odds(self,X):
        return np.dot(self.weight,X) + self.bias
    def sigmoid(self,z):
        return 1 / (1 + np.exp(-z))

    def train(self,X,y,X_val,y_val):
        n_sample,n_features=X.shape
        np.random.seed(42)
        self.weight=np.random.randn(X.shape[1])
        self.bias=np.random.randn()
        for i in range(self.epochs):
            indeces=np.random.permutation(n_sample)
            X=X.to_numpy()
            y=y.to_numpy()
            X_shuffle=X[indeces]
            y_shuffle=y[indeces]
            for j in range(0,batch_n_sample,self.batch_size):
                X_batch=X_shuffle[j:j+self.batch_size]
                y_batch=y_shuffle[j:j+self.batch_size]
                batch_size_current= len(X_batch)
                z=self.log_odds(X_batch)
                predict=self.sigmoid(z)
                error=predict - y_batch

                gradient_weight=(1/batch_size_current)*np.dot(error,X_batch.T)
                gradient_bias=(1/batch_size_current)*np.sum(error)
                
                self.weight -= lr * gradient_weight
                self.bias -= lr * gradient_bias

                loss= -np.mean(y_batch * np.log(predict + 1e-15) + (1 - y_batch)*np.log(1 - predict + 1e-15))
                self.loss_history.append(loss)

                z_val_set= np.dot(X_val.T,self.weight) + self.bias
                pred_val_set= 1 /(1 + np.exp(-z_val_set))

            val_error=pred_val_set - y_val
            val_loss= -np.mean(y_val * np.log(pred_val_set) + (1 - y_val + 1e-15)*np.log(1 - pred_val_set + 1e-15))
            self.val_loss_history.append(val_loss)

            print(f' the self.val_loss_history is {self.val_loss_history}')
            if self.best_loss is None or val_loss < self.best_loss - self.delta:
                self.best_loss = val_loss
                self.best_weight_v_set=np.copy(self.weight)
                self.best_bias_v_set=np.copy(self.bias)
                self.wait=0
            else:
                self.wait= +1
                if self.wait >= self.patience:
                    print(f' early stoping is at {i} epochs')
                    self.early_stop=True
                    break
        if self.best_weight_v_set is not None:
            self.best_weight_v_set=self.weight
            self.best_bias_v_set=self.bias
                
                
        
        

In [1]:
d=early_stoping()
d.train(X,y,X_val,y_val)

In [10]:
class logistic:
    def __init__(self,patience=6,epochs=1000,batchsize=30,delta=0.01,lr=0.01):
        self.patience=patience
        self.epochs=epochs
        self.batchsize=batchsize
        self.delta=delta
        self.lr=lr
        self.weight=None
        self.bias=None
        self.best_weight=None
        self.best_bias=None
        self.best_loss=None
        self.val_loss_history=[]
        self.train_loss_hist=[]
        self.wait=0
        self.early_stop=False
    def sigmoid(self,z):
        z=np.clip(z,-500,500)
        return 1/(1+np.exp(-z))
    def train(self,X,y,X_val,y_val):
        n_sample_train,n_feature_train=X.shape
        # n_sample_val,n_feature_val=X_val.shape

        np.random.seed(42)
        self.weight=np.random.randn(n_feature_train)
        self.bias=np.random.randn()
        X=(X-X.mean())/X.std()+1e-8
        X_val=(X_val-X_val.mean())/X_val.std()+1e-8

        for i in range(self.epochs):
            indices=np.random.permutation(len(y))
            X=X.to_numpy()
            y=y.to_numpy()
            X_shaffle=X[indices]
            y_shaffle=y[indices]
            for _ in range(0,len(y),self.batchsize):
                X_batch=X_shaffle[i:i+self.batchsize]
                y_batch=y_shaffle[i:i+self.batchsize]
                n_sample_batch=len(X_batch)

                z=np.dot(X_batch,self.weight)+ self.bias
                y_predi=self.sigmoid(z)
                error=y_predi-y_batch

                gradient_weight =(1/n_sample_batch)*np.dot(X_batch.T,error)
                gradient_bias  =(1/n_sample_batch) *np.sum(error)

                self.weight -= self.lr*gradient_weight
                self.bias -= self.lr*gradient_bias

                loss_train =-np.mean(y_batch*np.log(y_predi +1e-15) +(1-y_batch)*np.log(1-y_predi +1e-15))
                self.train_loss_hist.append(loss_train)

            z1=np.dot(X_val,self.weight) +self.bias
            y_predi_val=self.sigmoid(z1)
            loss_val = -np.mean(y_val*np.log(y_predi_val) +(1-y_val +1e-15)*np.log(1-y_predi_val +1e-15))
            self.val_loss_history.append(loss_val)

            if self.best_loss is None or loss_val < self.best_loss -self.delta:
                self.best_loss=loss_val
                self.best_weight =np.copy(self.weight)
                self.best_bias =np.copy(self.bias)
                self.wait=0
            else:
                self.wait +=1
                if self.wait >=self.patience:
                    print(f"early stopping at epoch {i}")
                    self.early_stop=True
                    break
        if self.best_weight is not  None:
            self.weight=self.best_weight
            self.bias=self.best_bias
            

In [2]:
r=logistic()
r.train(X,y,X_val,y_val)